# PredictHQ Forecasts API Example

This notebook demonstrates how to use PredictHQ’s Forecasts API. A sample demand dataset and configuration file is provided. This example will guide you through the process of making API calls and analyzing the results.

## Helpful Resources

- [Create an API token](https://docs.predicthq.com/getting-started/api-quickstart#create-an-access-token) if you do not already have one.
- Review the [Getting Started guide](https://docs.predicthq.com/getting-started/guides/forecasts-api-guides/getting-started) for more information on the Forecasts API.
- Review the [Understanding Forecast Metrics guide](https://docs.predicthq.com/getting-started/guides/forecasts-api-guides/understanding-forecast-accuracy-metrics) for more information on MAPE, MAE and RMSE.
- Review the [Troubleshooting guide](https://docs.predicthq.com/getting-started/guides/forecasts-api-guides/troubleshooting) for more information on common issues and how to resolve them.
- Check out the [API reference docs](https://docs.predicthq.com/api/forecasts) for more details on the Forecasts API endpoints and parameters.


In [ ]:
# Install dependencies if not already installed
# !pip install -r requirements.txt

In [ ]:
import os
import pandas as pd
import time
import json
import requests
import plotly.graph_objects as go
from dotenv import load_dotenv

load_dotenv()

## API Token

[Create an API token](https://docs.predicthq.com/getting-started/api-quickstart#create-an-access-token) if you do not already have one.

In [ ]:
PHQ_API_TOKEN = os.getenv("PHQ_API_TOKEN") or "REPLACE_WITH_YOUR_ACCESS_TOKEN"
API_URL = "https://api.predicthq.com"

headers = {
    "Authorization": f"Bearer {PHQ_API_TOKEN}",
    "Content-Type": "application/json"
}

## Create a Saved Location

Create a Saved Location for the location you want to forecast. This allows you to reference the location easily and ensures consistency across forecasts. The ideal radius for the location can be calculated using the Suggested Radius API.

In [ ]:
# Get location details
with open("data/sample_config.json", "r") as json_file:
    config = json.load(json_file)

print(config)

In [ ]:
# Get suggested radius
response = requests.get(
    url=f"{API_URL}/v1/suggested-radius/",
    headers=headers,
    params={
        "location.origin": f"{config['lat']},{config['lon']}",
        "industry": config["industry"],
        "radius_unit": "mi",
    },
)

data = response.json()
radius = data["radius"]
radius_unit = data["radius_unit"]

print(f"Suggested radius: {radius} {radius_unit}")

In [ ]:
# Create saved location
response = requests.post(
    url=f"{API_URL}/v1/saved-locations",
    headers=headers,
    json={
            "name": config["name"],
            "geojson": {
                "type": "Feature",
                "properties": {"radius": radius, "radius_unit": radius_unit},
                "geometry": {
                    "type": "Point",
                    "coordinates": [config["lon"], config["lat"]],
                },
            },
        }
)

location_id = response.json()["location_id"]
print(f"Saved Location ID: {location_id}")

## Define and Train a Forecast Model

In [ ]:
# Define model
response = requests.post(
    url=f"{API_URL}/v1/forecasts/models",
    headers=headers,
    json={
        "name": f"{config['name']} Forecast",
        "location": {"saved_location_id": location_id},
        "algo": "phq-xgboost",
        "forecast_window": "7d",
        "demand_type": {
            "industry": config["industry"],
        },
    },
)

model_id = response.json()["model_id"]
print(f"Model ID: {model_id}")

In [ ]:
# Upload demand
sample_demand_df = pd.read_csv("data/sample_demand.csv")
sample_demand_json = sample_demand_df.to_json(orient="records")

response = requests.post(
    url=f"{API_URL}/v1/forecasts/models/{model_id}/demand",
    headers=headers,
    json={"demand": json.loads(sample_demand_json)},
)

print(f"Demand upload: {'Successful' if response.status_code == 201 else 'Failed'}")

In [ ]:
# Train model
response = requests.post(
    url=f"{API_URL}/v1/forecasts/models/{model_id}/train",
    headers=headers,
)

print(f"Model training: {'Triggered' if response.status_code == 204 else 'Failed'}")

## Check Model Status

The model training may take up to a few minutes. Make sure the model is `ready` before proceeding.

In [ ]:
while True:
    response = requests.get(
        url=f"{API_URL}/v1/forecasts/models/{model_id}",
        headers=headers,
    )
    if response.status_code != 200:
        raise Exception(f"Failed to get model status: {response.status_code}, {response.text}")
    
    model_status = response.json()["model"]["readiness"]["status"]
    if model_status == "ready":
        print("Model is ready!")
        break
    if model_status == "failed":
        raise Exception("Model training failed")

    print(f"Model is {model_status}. Checking again in 30 seconds...")
    time.sleep(30)

## Evaluate Forecast Model

Use evaluation metrics such as MAPE to compare the model performance to other models, benchmarks, etc.

In [ ]:
# Get evaluation results
response = requests.get(
    url=f"{API_URL}/v1/forecasts/models/{model_id}",
    headers=headers,
)

metrics = response.json()['model']['metrics']
print(f"Evaluation metrics: {json.dumps(metrics, indent=2)}")

## Generate Forecasts

Get forecasted values for the next 7 days following the end of the uploaded demand data.

In [ ]:
# Get forecast
response = requests.get(
    url=f"{API_URL}/v1/forecasts/models/{model_id}/forecast",
    headers=headers,
    params={"date.gt": "2023-08-02"},
)

results = response.json()["results"]
forecasts_df = pd.DataFrame(results)

## Visualize results

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=sample_demand_df["date"],
        y=sample_demand_df["demand"],
        mode="lines+markers",
        name="Actual",
    )
)
fig.add_trace(
    go.Scatter(
        x=forecasts_df["date"],
        y=forecasts_df["forecast"],
        mode="lines+markers",
        name="Forecast",
    )
)
fig.update_layout(
    title="Forecasts for the next 7 days",
    xaxis_title="Date",
    yaxis_title="Demand",
)

fig

## Next Steps

- Review the [Getting Started guide](https://docs.predicthq.com/getting-started/guides/forecasts-api-guides/getting-started) for more information on the Forecasts API.
- Review the [Understanding Forecast Metrics guide](https://docs.predicthq.com/getting-started/guides/forecasts-api-guides/understanding-forecast-accuracy-metrics) for more information on MAPE, MAE and RMSE.
- Review the [Troubleshooting guide](https://docs.predicthq.com/getting-started/guides/forecasts-api-guides/troubleshooting) for more information on common issues and how to resolve them.
- Check out the [API reference docs](https://docs.predicthq.com/api/forecasts) for more details on the Forecasts API endpoints and parameters.

We haven't covered PHQ Explainability in this notebook. PHQ Explainability is crucial for understanding the model's predictions and ensuring transparency in the forecasting process. It provides you a list of the events impacting demand on a given day. Read more about explainability in the [Getting Started guide](https://docs.predicthq.com/getting-started/guides/forecasts-api-guides/getting-started#explainability).

Take the approach from this notebook and apply it to your own data and use cases.